## Wolffian duct differentiation - epithelium + mesenchyme

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr
import seaborn as sns
import scipy.stats
import anndata
import os

import scipy as scipy
import scipy as sp
import pickle as pkl
import matplotlib.pyplot as plt
import re
from collections import defaultdict
from statsmodels.nonparametric.smoothers_lowess import lowess
from numpy import asarray as ar
from collections import Counter
import networkx as nx
import igraph
import glob

In [ ]:
mese = sc.read('/nfs/team292/vl6/FetalReproductiveTract/wolffian_mese_late_post10pcw.h5ad')
print(mese.shape)
epi = sc.read('/nfs/team292/vl6/FetalReproductiveTract/wolffian_epi_late_post10pcw.h5ad')
print(epi.shape)

In [ ]:
# Reset to raw 
import anndata 
mese_raw = anndata.AnnData(X = mese.raw.X, var = mese.raw.var, obs = mese.obs)
epi_raw = anndata.AnnData(X = epi.raw.X, var = epi.raw.var, obs = epi.obs)
print(mese_raw.shape, epi_raw.shape)

In [ ]:
adata = mese_raw.concatenate(epi_raw, join='outer',index_unique=None)
adata.shape

In [ ]:
## import reproductive tract utils functions
import sys
cwd = '/nfs/team292/vl6/Experiments/Utils/'
sys.path.append(cwd)

import reptract_utils
import reptract_genes

In [ ]:
adata.raw = adata.copy()
sc.pp.filter_genes(adata, min_counts=10)
# adata = reptract_utils.per_gene_analysis(adata)
adata = reptract_utils.normalize_log_transform(adata)
adata = reptract_utils.hvgs_pca_umap(adata)

In [ ]:
sc.pl.umap(adata, color=['stage_pcw', 'sex', 
                         'n_genes', 'donor',  'doublet_scores',
                        'celltype'], ncols = 2, wspace = 0.3, 
          color_map = 'OrRd')

In [ ]:
sc.pl.umap(adata, color = ['PAX8', 'GATA3', 'MUC6', 'ESR1', 'VIL1', 'HOXA5', 'HOXA7', 'HOXA10', 'PTGER3', 'ACTA2', 'DES', 'MYH11',
                          'HOXA13', 'ISL1', 'TMEM163',  'WNT4', 'GDF10', 'COMP', 'CALY', 
                          'WNT2', 'SFRP2', 'CDH7', 'RAI2',  'HMGA2', 
                          'KRT18', 'HOXA11', 'HOXC10', 'SRD5A2', 'EMX2',  
                          'DLK1'], wspace = 0.15, use_raw = False,
           color_map = 'OrRd', ncols = 2)

### Harmony integration

In [ ]:
sc.set_figure_params(scanpy=True, dpi=80, dpi_save=150, frameon=True, vector_friendly=True, 
                         fontsize=14, figsize=(6,6), color_map=None, format='pdf', facecolor=None, transparent=False)

In [ ]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [ ]:
import rpy2.rinterface_lib.callbacks
import logging
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
# Extract PCA matrix and batch array
n_pcs = 25 # Principal components used to perform dimensionality reduction 
pca = adata.obsm['X_pca'][:, 0:(n_pcs)]
batch = adata.obs['donor'] # Batch ID, in this case stage

In [ ]:
%%R -i pca -i batch -o hem

library(harmony)
library(magrittr)
set.seed(1000)
hem <- HarmonyMatrix(pca, batch, theta=0, lambda = 6, do_pca=FALSE, verbose = FALSE)  
hem = data.frame(hem)

In [ ]:
# Add harmony values to the anndata object --> overwriting PCA 
adata.obsm['X_pca_harmony'] = hem.values 

In [ ]:
sc.pp.neighbors(adata, n_pcs = n_pcs, use_rep = 'X_pca_harmony', random_state= 123, key_added = 'harmony')


In [ ]:
sc.tl.umap(adata, min_dist=0.6, spread = 0.8, neighbors_key = 'harmony')


In [ ]:
sc.pl.umap(adata, color = [ 'PAX8', 'GATA3', 'MUC6', 'ESR1', 'VIL1', 'HOXA5', 'HOXA7', 'HOXA10', 'PTGER3', 'ACTA2', 'DES', 'MYH11',
                          'HOXA13', 'ISL1', 'TMEM163',  'WNT4', 'GDF10', 'COMP', 'CALY', 
                          'WNT2', 'SFRP2', 'CDH7', 'RAI2',  'HMGA2', 
                          'KRT18', 'HOXA11', 'HOXC10', 'SRD5A2', 'EMX2',  
                          'DLK1'], wspace = 0.15, use_raw = False,
           color_map = 'OrRd', ncols = 2)

In [ ]:
sc.pl.umap(adata, color=['stage_pcw', 'sex', 
                         'n_genes', 'donor',  'doublet_scores', 'celltype'], ncols = 2, wspace = 0.3, 
          color_map = 'OrRd')

In [ ]:
sc.pl.umap(adata, color = [ 'mese_wolffian_lowres', 'epi_wolffian_lowres'], wspace = 0.15, use_raw = False,
           color_map = 'OrRd', ncols = 2)

In [ ]:
adata.obs['mese_wolffian_lowres'] = adata.obs['mese_wolffian_lowres'].astype(str)

In [ ]:
adata.obs['wolffian_lowres'] = np.where(adata.obs['mese_wolffian_lowres'] == 'nan',
                                         adata.obs['epi_wolffian_lowres'], adata.obs['mese_wolffian_lowres'])

In [ ]:
sc.pl.umap(adata, color = [ 'wolffian_lowres'], wspace = 0.15, use_raw = False,
           color_map = 'OrRd', ncols = 2)

In [ ]:
sc.pl.umap(adata, color = 'wolffian_lowres', 
          palette = ['lightskyblue', 'steelblue', 'mediumturquoise', 'cornflowerblue'], 
          save = '_wolffian_lowres')

In [ ]:
adata.write('/nfs/team292/vl6/FetalReproductiveTract/wolffian_mese_epi_late_post10pcw.h5ad')

In [ ]:
adata = sc.read('/nfs/team292/vl6/FetalReproductiveTract/wolffian_mese_epi_late_post10pcw.h5ad')

In [ ]:
sc.pl.umap(adata, color = ['HOXC6', 'FOXC1', 'HOXC4', 'HOXA5', 'FOS', 'HOXA10', 'EGR1', 'HOXC8', 'FOSB', 'JUNB', 
                           'HOXA9', 'HOXA7', 'HOXA11', 'HOXC5', 'SOX4', 'RORB', 'EGR3', 'JUN', 'HOXA6', 'OSR2', 
                           'TRPS1', 'MAFB', 'FOXO4', 
                           'CUX2', 'HOXA13', 'NFE2L2', 'TEAD3', 'ARX', 'HOXD11'], color_map = 'OrRd', 
          ncols = 2, use_raw = False)

In [ ]:
sc.pl.umap(adata, color = ['IGFBP3', 'TMEM219', 'CDH3', 'CDH7', 'COL13A1', 'ITGB1', 'ITGA1', 
                          'ITGA2', 'ITGA10', 'ITGA11', 'PPARA', 'PPARD', 'PPARG', 'SULT1E1', 
                          'EDN3', 'EDNRA', 'EDNRB', 'NTN1', 'DCC', 'UNC5B', 'UNC5C', 
                          'NLGN1', 'NRXN1', 'NRXN3', 'NRXN2', 'DKK2', 'KREMEN2', 'LRP6', 
                          'GDNF', 'RET', 'GFRA1', 'TNC', 'NELL2', 'ROBO3', 'RSPO2', 'LGR4', 'LGR5', 'GDF7', 'ANGPT1', 'TEK'], color_map = 'OrRd', 
          ncols = 2, use_raw = False)

In [ ]:
sc.pl.umap(adata, color = ['SFRP1', 'WIF1',  'WNT6', 'WNT4', 'WNT3', 'WNT5A', 'WNT11', 
                           'WNT9A', 'WNT9B', 'WNT2', 'WNT2B', 'SLC1A1', 'GLS', 'SLC1A3', 
                           'SLC1A6',  'GRM7', 'ADRA2A', 'PNMT', 'SCARA5', 'FTH1', 'FTL', 
                          'GABBR2', 'SLC6A12', 'GAD1', 'ALDH1A1', 'RORB', 'AR', 'ESR1', 'HNF1A', 'PITX2', 'MLXIPL'], color_map = 'OrRd', 
          ncols = 2, use_raw = False)

## ATAC visualisations - MESENCHYME

In [ ]:
outDir = '/lustre/scratch126/cellgen/team292/vl6/pycistopic/wolffian_mese_post9pcw/'

In [ ]:
atac_annots = pd.read_csv(outDir + 'wolff_mese_embedding.csv', index_col = 0)
atac_annots.head()

In [ ]:
atac_annots.shape

In [ ]:
# Create fake matrix 
fake_matrix = np.zeros([2434, 20000])
fake_matrix.shape

In [ ]:
fake_vars = pd.DataFrame({'n_genes' : [5] * 20000})

In [ ]:
import anndata 
adata = anndata.AnnData(X = fake_matrix, var = fake_vars, obs = atac_annots)

In [ ]:
adata.obsm['X_umap'] = atac_annots[['tsne1', 'tsne2']].to_numpy()

In [ ]:
sc.pl.umap(adata, color = 'stage', cmap = 'viridis', save = '_atac_wolff_mese_stage')

In [ ]:
color_dict = {'EpididymisMese': 'orange', 'VasDeferensMese': 'orangered'}

In [ ]:
sc.pl.umap(adata, color = 'mese_wolffian_lowres', 
           palette = color_dict, save = '_atac_wolff_mese_celltype')

## ATAC visualisations - EPITHELIUM

In [ ]:
outDir2 = '/lustre/scratch126/cellgen/team292/vl6/pycistopic/wolffian_epi_post9pcw/'


In [ ]:
atac_annots = pd.read_csv(outDir2 + 'wolff_epi_embedding.csv', index_col = 0)
atac_annots.head()

In [ ]:
atac_annots.shape

In [ ]:
# Create fake matrix 
fake_matrix = np.zeros([1213, 20000])
fake_matrix.shape

In [ ]:
fake_vars = pd.DataFrame({'n_genes' : [5] * 20000})

In [ ]:
import anndata 
adata = anndata.AnnData(X = fake_matrix, var = fake_vars, obs = atac_annots)

In [ ]:
adata.obsm['X_umap'] = atac_annots[['tsne1', 'tsne2']].to_numpy()

In [ ]:
sc.set_figure_params(scanpy=True, dpi=80, dpi_save=150, 
                         frameon=True, vector_friendly=True, fontsize=14, figsize=[7,7], color_map=None, 
                         format='pdf', facecolor=None, transparent=False)

In [ ]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [ ]:
sc.pl.umap(adata, color = 'stage', cmap = 'viridis', save = '_atac_wolff_epi_stage')

In [ ]:
color_dict = {'EpididymisEpi': 'goldenrod',
 'VasDeferensEpi': 'indianred',
 'UpperVaginaEpi': 'mediumorchid'}

In [ ]:
sc.pl.umap(adata, color = 'epi_wolffian_lowres', 
           palette = color_dict, save = '_atac_wolff_epi_celltype')